# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df

,city,country,lat,lon,temp,humidity,cloudiness,wind speed
0,Tucuruí,BR,-3.77,-49.67,91.92,35,1,2.51
1,Barrow,US,71.29,-156.79,42.80,81,90,23.04
2,Mataura,NZ,-46.19,168.86,48.38,87,100,5.17
3,Rabo de Peixe,PT,37.80,-25.58,73.40,88,40,13.87
4,Tiksi,RU,71.69,128.87,39.45,86,84,7.92
...,...,...,...,...,...,...,...,...
576,Diego de Almagro,CL,-26.37,-70.05,79.12,15,100,11.12
577,Mehamn,NO,71.04,27.85,59.00,72,0,3.36
578,São João da Barra,BR,-21.64,-41.05,78.91,65,20,16.11
579,Fairbanks,US,64.84,-147.72,55.35,82,90,6.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_data_df[["lat", "lon"]]
humidity = city_data_df["humidity"]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=300, point_radius=3)
fig.add_layer(heatmap_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
perf_city_df = city_data_df.loc[(city_data_df["temp"] < 85)  
                              & (city_data_df["temp"] > 72)
                              & (city_data_df["wind speed"] < 5) 
                              & (city_data_df["cloudiness"] == 0)].dropna()
perf_city_df

,city,country,lat,lon,temp,humidity,cloudiness,wind speed
25,Jaca,ES,42.57,-0.55,84.94,41,0,2.89
106,Portland,US,45.52,-122.68,77.61,52,0,1.99
115,Kavieng,PG,-2.57,150.80,81.79,69,0,4.99
123,Barcelos,PT,41.54,-8.62,81.59,64,0,4.70
259,Cassino,IT,41.49,13.83,81.00,79,0,1.99
403,Mayskiy,RU,47.69,40.10,73.40,49,0,2.24
431,Brus,RS,43.38,21.03,82.40,61,0,2.24
524,Linfen,CN,36.09,111.52,78.98,72,0,3.02
537,Carutapera,BR,-1.20,-46.02,84.11,68,0,4.56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = perf_city_df[["city", "country","lat", "lon"]]
hotel_df["hotel name"] = ""

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}


for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lon = row["lon"]
    
    params["location"] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    response = response.json()
    
    try:
        hotel_df.loc[index, "hotel name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("missing")

hotel_df


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,city,country,lat,lon,hotel name
25,Jaca,ES,42.57,-0.55,Hotel Real Jaca & Spa
106,Portland,US,45.52,-122.68,Portland Marriott City Center
115,Kavieng,PG,-2.57,150.80,Nusa Island Retreat
123,Barcelos,PT,41.54,-8.62,Hotel Do Terço
259,Cassino,IT,41.49,13.83,Hotel La Pace - Experience
403,Mayskiy,RU,47.69,40.10,Dio Lakrua
431,Brus,RS,43.38,21.03,Vila Djordjevic
524,Linfen,CN,36.09,111.52,Suoxing Hotel
537,Carutapera,BR,-1.20,-46.02,Lidera


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))